In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/rd/miniconda3/envs/dsenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
strings_to_tokenize = [
    "Hello, world!",
    "This is a test string.",
    "Transformers are amazing for NLP tasks.",
    "Let's see how many tokens these strings take.",
]
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

In [7]:
ids = [
    tokenizer.encode(string) for string in strings_to_tokenize
]

[[9707, 11, 1879, 0],
 [1986, 374, 264, 1273, 914, 13],
 [8963, 388, 525, 7897, 369, 451, 12567, 9079, 13],
 [10061, 594, 1490, 1246, 1657, 11211, 1493, 9069, 1896, 13]]

In [10]:
padded = tokenizer.pad({"input_ids": ids}, padding="longest", return_tensors="pt")

In [13]:
padded

{'input_ids': tensor([[  9707,     11,   1879,      0, 151643, 151643, 151643, 151643, 151643,
         151643],
        [  1986,    374,    264,   1273,    914,     13, 151643, 151643, 151643,
         151643],
        [  8963,    388,    525,   7897,    369,    451,  12567,   9079,     13,
         151643],
        [ 10061,    594,   1490,   1246,   1657,  11211,   1493,   9069,   1896,
             13]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
import torch
input_ids = padded["input_ids"]
attn_mask = padded["attention_mask"]
with torch.no_grad():
    logits = model(input_ids=input_ids, attention_mask=attn_mask).logits


In [16]:
input_ids.shape

torch.Size([4, 10])

In [18]:
logits

tensor([[[ 7.2801,  5.6999,  0.6545,  ..., -3.7292, -3.7295, -3.7291],
         [ 8.3219, -1.3552,  0.0493,  ..., -2.5598, -2.5601, -2.5597],
         [18.1119,  8.2721,  7.2261,  ..., -0.6242, -0.6245, -0.6243],
         ...,
         [ 9.9240, 10.3055, 17.2309,  ..., -2.0765, -2.0767, -2.0766],
         [ 9.8248,  9.7491, 15.4928,  ..., -1.9129, -1.9130, -1.9129],
         [ 9.4980, 10.3333, 15.4334,  ..., -1.7121, -1.7124, -1.7121]],

        [[ 4.9903,  3.1928,  1.8066,  ..., -2.5430, -2.5436, -2.5429],
         [ 5.0394,  1.7928,  0.1334,  ..., -2.1265, -2.1262, -2.1263],
         [ 7.6685,  4.6036,  1.2015,  ..., -1.7160, -1.7166, -1.7159],
         ...,
         [11.1904, 13.1078, 13.6556,  ..., -0.8852, -0.8855, -0.8852],
         [ 9.6250, 10.4838, 11.2969,  ..., -1.2635, -1.2636, -1.2636],
         [ 9.0981,  9.5217, 11.5075,  ..., -2.3557, -2.3558, -2.3558]],

        [[ 3.6877,  5.9022,  1.8271,  ..., -2.1206, -2.1204, -2.1206],
         [ 8.4649,  9.3085,  6.2698,  ..., -0

In [19]:
torch.softmax(logits, dim=-1)

tensor([[[7.6809e-04, 1.5819e-04, 1.0185e-06,  ..., 1.2710e-08,
          1.2707e-08, 1.2712e-08],
         [1.7020e-04, 1.0672e-08, 4.3472e-08,  ..., 3.1997e-09,
          3.1985e-09, 3.1997e-09],
         [3.7424e-01, 1.9943e-05, 7.0070e-06,  ..., 2.7302e-09,
          2.7292e-09, 2.7300e-09],
         ...,
         [2.1800e-04, 3.1926e-04, 3.2493e-01,  ..., 1.3388e-09,
          1.3385e-09, 1.3387e-09],
         [3.1530e-04, 2.9232e-04, 9.1264e-02,  ..., 2.5184e-09,
          2.5179e-09, 2.5184e-09],
         [2.1285e-04, 4.9074e-04, 8.0499e-02,  ..., 2.8812e-09,
          2.8805e-09, 2.8812e-09]],

        [[4.9369e-06, 8.1810e-07, 2.0454e-07,  ..., 2.6410e-09,
          2.6394e-09, 2.6413e-09],
         [8.7292e-07, 3.3964e-08, 6.4612e-09,  ..., 6.7433e-10,
          6.7450e-10, 6.7446e-10],
         [1.6880e-05, 7.8760e-07, 2.6230e-08,  ..., 1.4182e-09,
          1.4175e-09, 1.4183e-09],
         ...,
         [1.1690e-04, 7.9528e-04, 1.3755e-03,  ..., 6.6596e-10,
          6.657

In [22]:
torch.softmax(logits, dim=-1).gather(-1, input_ids[:,:1].unsqueeze(-1))

tensor([[[3.1165e-06]],

        [[1.2550e-06]],

        [[3.1799e-06]],

        [[5.0264e-06]]])

In [23]:
new_lp = torch.tensor([
  [1,2,3],
  [4,5,6],
  [7,8,9],
  [10,11,12]
])

In [ ]:
advantages = torch.tensor([1, 2, 3, 4])
advantages[:, None].expand_as()